In [1]:
import pandas as pd

In [2]:
prinstine = pd.read_csv('prinstine_data.csv', index_col = 'id')

# Augment by sub <-> obj with at least one wrong answer

In [3]:
se = prinstine.apply(lambda x : x['object_entity'] if (x['counter'] != 0 and (x['label'] == 'no_relation' or x['label'] == 'org:alternate_names' or x['label'] == 'per:alternate_names' or x['label'] == 'per:colleagues' or x['label'] == 'per:siblings' or x['label'] == 'per:spouse' or x['label'] == 'per:other_family')) else 0, axis = 1)
oe = prinstine.apply(lambda x : x['subject_entity'] if (x['counter'] != 0 and (x['label'] == 'no_relation' or x['label'] == 'org:alternate_names' or x['label'] == 'per:alternate_names' or x['label'] == 'per:colleagues' or x['label'] == 'per:siblings' or x['label'] == 'per:spouse' or x['label'] == 'per:other_family')) else 0, axis = 1)

In [4]:
sub_to_obj = prinstine.copy()
sub_to_obj['subject_entity'] = se
sub_to_obj['object_entity'] = oe
sub_to_obj = sub_to_obj[sub_to_obj['subject_entity'] != 0]
print(len(sub_to_obj))

4136


# Augment sub <_> obj with at least one wrong answer with label change

### org:members < = > org:menbers_of

In [5]:
se = prinstine.apply(lambda x : x['object_entity'] if (x['counter'] != 0 and (x['label'] == 'org:members')) else 0, axis = 1)
oe = prinstine.apply(lambda x : x['subject_entity'] if (x['counter'] != 0 and (x['label'] == 'org:members')) else 0, axis = 1)

In [6]:
mberof = prinstine.copy()
mberof['subject_entity'] = se
mberof['object_entity'] = oe
mberof = mberof[mberof['subject_entity'] != 0]
mberof['label'] = 'org:member_of'
print(len(mberof))

245


In [7]:
se = prinstine.apply(lambda x : x['object_entity'] if (x['counter'] != 0 and (x['label'] == 'org:member_of')) else 0, axis = 1)
oe = prinstine.apply(lambda x : x['subject_entity'] if (x['counter'] != 0 and (x['label'] == 'org:member_of')) else 0, axis = 1)

In [8]:
mber = prinstine.copy()
mber['subject_entity'] = se
mber['object_entity'] = oe
mber = mber[mber['subject_entity'] != 0]
mber['label'] = 'org:members'
print(len(mber))

566


### per:parents < = > per:children

In [9]:
se = prinstine.apply(lambda x : x['object_entity'] if (x['counter'] != 0 and (x['label'] == 'per:parents')) else 0, axis = 1)
oe = prinstine.apply(lambda x : x['subject_entity'] if (x['counter'] != 0 and (x['label'] == 'per:parents')) else 0, axis = 1)

In [10]:
parent = prinstine.copy()
parent['subject_entity'] = se
parent['object_entity'] = oe
parent = parent[parent['subject_entity'] != 0]
parent['label'] = 'per:children'
print(len(parent))

183


In [11]:
se = prinstine.apply(lambda x : x['object_entity'] if (x['counter'] != 0 and (x['label'] == 'per:children')) else 0, axis = 1)
oe = prinstine.apply(lambda x : x['subject_entity'] if (x['counter'] != 0 and (x['label'] == 'per:children')) else 0, axis = 1)

In [12]:
child = prinstine.copy()
child['subject_entity'] = se
child['object_entity'] = oe
child = child[child['subject_entity'] != 0]
child['label'] = 'per:children'
print(len(child))

222


In [13]:
se = prinstine.apply(lambda x : x['object_entity'] if (x['counter'] != 0 and (x['label'] == 'org:top_members/employees')) else 0, axis = 1)
oe = prinstine.apply(lambda x : x['subject_entity'] if (x['counter'] != 0 and (x['label'] == 'org:top_members/employees')) else 0, axis = 1)

In [14]:
top_em = prinstine.copy()
top_em['subject_entity'] = se
top_em['object_entity'] = oe
top_em = top_em[top_em['subject_entity'] != 0]
top_em['label'] = 'per:employee_of'
print(len(top_em))

343


In [15]:
cleaned_augmented = pd.concat([prinstine, top_em, child, parent, mber, mberof, sub_to_obj]) 

In [16]:
cleaned_augmented = cleaned_augmented.loc[:,['sentence', 'subject_entity', 'object_entity', 'label', 'source']]
cleaned_augmented = cleaned_augmented.drop_duplicates()

In [17]:
cleaned_augmented.to_csv('cleaned_target_augmented.csv')

In [18]:
(len(cleaned_augmented) - len(prinstine)) / len(prinstine)

0.17005921539600297